In [2]:
# Importy podstawowe
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv
import os

# Importy do similarity
from sklearn.metrics.pairwise import cosine_similarity

# Import naszej funkcji do PDF
import sys
sys.path.append('..')  # Żeby móc importować z utils/
from utils.pdf_handler import extract_text_from_pdf

# Załaduj zmienne środowiskowe
load_dotenv()

print("✅ Importy załadowane!")

✅ Importy załadowane!


In [3]:
# Inicjalizacja klienta OpenAI
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    print("❌ Brak klucza OPENAI_API_KEY w pliku .env!")
else:
    openai_client = OpenAI(api_key=api_key)
    print("✅ Klient OpenAI zainicjalizowany!")

# Wybór modelu embeddingu
embedding_model = "text-embedding-3-small"  # Tańszy, 1536 wymiarów
# embedding_model = "text-embedding-3-large"  # Droższy, 3072 wymiary

print(f"📊 Model: {embedding_model}")

✅ Klient OpenAI zainicjalizowany!
📊 Model: text-embedding-3-small


In [4]:
# Funkcja do generowania embeddingu
def calculate_embedding(text, model=embedding_model):
    """
    Generuje embedding dla podanego tekstu.
    
    Args:
        text (str): Tekst do przekształcenia w embedding
        model (str): Model OpenAI (domyślnie: embedding_model)
    
    Returns:
        np.array: Wektor embeddingu lub None przy błędzie
    """
    
    try:
        result = openai_client.embeddings.create(
            input=[text],
            model=model,
        )
        
        # Zamień na numpy array
        embedding = np.array(result.data[0].embedding)
        
        return embedding
    
    except Exception as e:
        print(f"❌ Błąd podczas generowania embeddingu: {e}")
        return None


# Test funkcji
print("🧪 TEST FUNKCJI calculate_embedding:")
print("=" * 60)

test_text = "Python developer with 5 years experience"
test_embedding = calculate_embedding(test_text)

if test_embedding is not None:
    print(f"✅ Embedding wygenerowany!")
    print(f"📊 Wymiary: {len(test_embedding)}")
    print(f"📊 Pierwsze 5 wartości: {test_embedding[:5]}")
    print(f"📊 Typ: {type(test_embedding)}")
else:
    print("❌ Nie udało się wygenerować embeddingu")

🧪 TEST FUNKCJI calculate_embedding:
✅ Embedding wygenerowany!
📊 Wymiary: 1536
📊 Pierwsze 5 wartości: [-0.02305795 -0.00878863  0.00386639  0.01854157  0.01050974]
📊 Typ: <class 'numpy.ndarray'>


In [5]:
# Ścieżka do Twojego CV
cv_path = r"C:\Users\slast\PYTHON\0_projekty do portfolio\16_Embeddings(cv_job_matching)_AZURE\cv-job-matcher\data\cv_Sławomir_Strzelec.pdf"

print("📄 GENEROWANIE EMBEDDINGU DLA CV")
print("=" * 60)

# Wyciągnij tekst z CV
cv_text = extract_text_from_pdf(cv_path, method='pdfplumber', clean=True)

if cv_text:
    print(f"✅ Tekst wyekstraktowany: {len(cv_text)} znaków")
    print(f"📝 Fragment CV:\n{cv_text[:200]}...\n")
    
    # Wygeneruj embedding dla CV
    cv_embedding = calculate_embedding(cv_text)
    
    if cv_embedding is not None:
        print(f"✅ Embedding CV wygenerowany!")
        print(f"📊 Wymiary: {len(cv_embedding)}")
        print(f"📊 Norma wektora: {np.linalg.norm(cv_embedding):.6f}")
        print(f"   (powinno być ~1.0 dla embeddingów OpenAI)")
    else:
        print("❌ Nie udało się wygenerować embeddingu CV")
else:
    print("❌ Nie udało się wyekstraktować tekstu z CV")

📄 GENEROWANIE EMBEDDINGU DLA CV
✅ Tekst wyekstraktowany: 9668 znaków
📝 Fragment CV:
SŁAWOMIR STRZELEC Machine Learning Engineer | Python & PyTorch | Time Series & Predictive Modeling sla.strzelec@gmail.com | +48 732 022 909 Kraków, Poland | LinkedIn | GitHub | Portfolio PROFESSIONAL ...

✅ Embedding CV wygenerowany!
📊 Wymiary: 1536
📊 Norma wektora: 1.000000
   (powinno być ~1.0 dla embeddingów OpenAI)


In [6]:
# Przykładowe oferty pracy (hardcoded do testów)
sample_jobs = [
    {
        "id": 1,
        "title": "Senior Machine Learning Engineer",
        "company": "TechCorp",
        "description": "Looking for ML Engineer with Python, PyTorch, scikit-learn. Experience with time series forecasting and deploying models to production. 5+ years experience required.",
        "requirements": "Python, PyTorch, scikit-learn, time series, production ML",
        "location": "Kraków, Poland",
        "link": "https://example.com/job1"
    },
    {
        "id": 2,
        "title": "Data Scientist",
        "company": "FinanceAI",
        "description": "Data Scientist position focusing on statistical modeling and business intelligence. SQL, Python, Tableau required. No ML experience needed.",
        "requirements": "Python, SQL, Tableau, statistics, business intelligence",
        "location": "Warsaw, Poland",
        "link": "https://example.com/job2"
    },
    {
        "id": 3,
        "title": "Python Backend Developer",
        "company": "WebDev Inc",
        "description": "Backend developer for Django/Flask applications. REST APIs, databases, Docker. No ML/AI required.",
        "requirements": "Python, Django, Flask, REST API, PostgreSQL, Docker",
        "location": "Remote",
        "link": "https://example.com/job3"
    },
    {
        "id": 4,
        "title": "AI Research Engineer",
        "company": "DeepMind Clone",
        "description": "Research position in deep learning. PyTorch, TensorFlow, transformers, NLP. PhD or MSc required. Publishing in top venues.",
        "requirements": "PyTorch, TensorFlow, transformers, NLP, research, PhD",
        "location": "London, UK",
        "link": "https://example.com/job4"
    },
    {
        "id": 5,
        "title": "Junior Python Developer",
        "company": "StartupXYZ",
        "description": "Entry-level Python position. Basic programming, learning on the job. No experience required.",
        "requirements": "Python basics, willingness to learn",
        "location": "Gdańsk, Poland",
        "link": "https://example.com/job5"
    }
]

print("📋 PRZYKŁADOWE OFERTY PRACY")
print("=" * 60)
print(f"Liczba ofert: {len(sample_jobs)}")
print("\n")

for job in sample_jobs:
    print(f"🔹 [{job['id']}] {job['title']} - {job['company']}")
    print(f"   📍 {job['location']}")
    print()

📋 PRZYKŁADOWE OFERTY PRACY
Liczba ofert: 5


🔹 [1] Senior Machine Learning Engineer - TechCorp
   📍 Kraków, Poland

🔹 [2] Data Scientist - FinanceAI
   📍 Warsaw, Poland

🔹 [3] Python Backend Developer - WebDev Inc
   📍 Remote

🔹 [4] AI Research Engineer - DeepMind Clone
   📍 London, UK

🔹 [5] Junior Python Developer - StartupXYZ
   📍 Gdańsk, Poland



In [7]:
# Generowanie embeddingów dla wszystkich ofert
print("🔄 GENEROWANIE EMBEDDINGÓW DLA OFERT PRACY")
print("=" * 60)

# Dla każdej oferty łączymy title + description + requirements
for job in sample_jobs:
    # Połącz wszystkie informacje w jeden tekst
    job_text = f"{job['title']}. {job['description']} Requirements: {job['requirements']}"
    
    # Wygeneruj embedding
    job_embedding = calculate_embedding(job_text)
    
    if job_embedding is not None:
        # Zapisz embedding w słowniku oferty
        job['embedding'] = job_embedding
        print(f"✅ [{job['id']}] {job['title'][:40]:<40} - embedding wygenerowany")
    else:
        print(f"❌ [{job['id']}] {job['title']} - błąd!")

print("\n✅ Wszystkie embeddingi wygenerowane!")

🔄 GENEROWANIE EMBEDDINGÓW DLA OFERT PRACY
✅ [1] Senior Machine Learning Engineer         - embedding wygenerowany
✅ [2] Data Scientist                           - embedding wygenerowany
✅ [3] Python Backend Developer                 - embedding wygenerowany
✅ [4] AI Research Engineer                     - embedding wygenerowany
✅ [5] Junior Python Developer                  - embedding wygenerowany

✅ Wszystkie embeddingi wygenerowane!


In [8]:
# Obliczanie podobieństwa między CV a ofertami
print("🎯 OBLICZANIE PODOBIEŃSTWA CV DO OFERT")
print("=" * 60)

# Dla każdej oferty oblicz cosine similarity
for job in sample_jobs:
    # Oblicz podobieństwo
    similarity = cosine_similarity([cv_embedding], [job['embedding']])[0][0]
    
    # Zapisz w słowniku
    job['similarity'] = similarity
    
    # Kolorowanie wyniku
    if similarity > 0.8:
        color = "🟢"
        rating = "Świetne dopasowanie"
    elif similarity > 0.6:
        color = "🟠"
        rating = "Średnie dopasowanie"
    else:
        color = "🔴"
        rating = "Słabe dopasowanie"
    
    print(f"{color} [{job['id']}] {job['title'][:35]:<35} | {similarity*100:>5.2f}% | {rating}")

print("\n✅ Podobieństwa obliczone!")

🎯 OBLICZANIE PODOBIEŃSTWA CV DO OFERT
🔴 [1] Senior Machine Learning Engineer    | 43.87% | Słabe dopasowanie
🔴 [2] Data Scientist                      | 38.92% | Słabe dopasowanie
🔴 [3] Python Backend Developer            | 38.37% | Słabe dopasowanie
🔴 [4] AI Research Engineer                | 44.02% | Słabe dopasowanie
🔴 [5] Junior Python Developer             | 33.90% | Słabe dopasowanie

✅ Podobieństwa obliczone!


In [9]:
# DIAGNOZA - sprawdźmy długości tekstów
print("🔍 DIAGNOZA PROBLEMU")
print("=" * 60)

print(f"\n📄 Długość CV: {len(cv_text)} znaków")
print(f"📝 Fragment CV: {cv_text[:150]}...")

print("\n📋 Długości opisów ofert:")
for job in sample_jobs:
    job_text = f"{job['title']}. {job['description']} Requirements: {job['requirements']}"
    print(f"  [{job['id']}] {job['title'][:30]:<30} | {len(job_text):>4} znaków")

print("\n💡 PROBLEM ZIDENTYFIKOWANY:")
print("   CV jest ~40x dłuższe niż opisy ofert!")
print("   Embeddingi porównują CAŁĄ semantykę, nie tylko keywords.")

🔍 DIAGNOZA PROBLEMU

📄 Długość CV: 9668 znaków
📝 Fragment CV: SŁAWOMIR STRZELEC Machine Learning Engineer | Python & PyTorch | Time Series & Predictive Modeling sla.strzelec@gmail.com | +48 732 022 909 Kraków, Po...

📋 Długości opisów ofert:
  [1] Senior Machine Learning Engine |  271 znaków
  [2] Data Scientist                 |  225 znaków
  [3] Python Backend Developer       |  189 znaków
  [4] AI Research Engineer           |  212 znaków
  [5] Junior Python Developer        |  167 znaków

💡 PROBLEM ZIDENTYFIKOWANY:
   CV jest ~40x dłuższe niż opisy ofert!
   Embeddingi porównują CAŁĄ semantykę, nie tylko keywords.


In [10]:
# Ekstrakcja sekcji Skills z CV
print("🔧 EKSTRAKCJA SEKCJI SKILLS")
print("=" * 60)

# Wyświetl fragment CV żeby zobaczyć strukturę
print("📄 Początek CV (pierwsze 1000 znaków):")
print("-" * 60)
print(cv_text[:1000])
print("-" * 60)

# Sprawdź czy są słowa kluczowe
keywords = ["SKILLS", "TECHNICAL SKILLS", "TECHNOLOGIES", "TOOLS"]
print(f"\n🔍 Szukam słów kluczowych: {keywords}")

for keyword in keywords:
    if keyword in cv_text.upper():
        position = cv_text.upper().find(keyword)
        print(f"  ✅ Znaleziono '{keyword}' na pozycji {position}")
    else:
        print(f"  ❌ Nie znaleziono '{keyword}'")

🔧 EKSTRAKCJA SEKCJI SKILLS
📄 Początek CV (pierwsze 1000 znaków):
------------------------------------------------------------
SŁAWOMIR STRZELEC Machine Learning Engineer | Python & PyTorch | Time Series & Predictive Modeling sla.strzelec@gmail.com | +48 732 022 909 Kraków, Poland | LinkedIn | GitHub | Portfolio PROFESSIONAL SUMMARY Machine Learning Engineer with strong hands-on experience in Python, PyTorch and scikit- learn, focused on building, validating and deploying predictive models and time series solutions. Skilled in processing large-scale, noisy datasets (4M+ records), developing reproducible ETL/ML pipelines, time series forecasting (Prophet, ARIMA), anomaly detection and statistical modeling. Experienced in model evaluation, hyperparameter tuning, robustness checks and production deployment using MLflow, Docker basics and AWS SageMaker (certified 2025). Proven ability to translate complex analytical insights into actionable outcomes in Agile environments, collaborating clos

In [11]:
# Ekstrakcja skills używając OpenAI
print("🤖 EKSTRAKCJA SKILLS PRZEZ OpenAI")
print("=" * 60)

# Prompt dla OpenAI
extraction_prompt = f"""
Przeanalizuj poniższe CV i wyciągnij TYLKO najważniejsze umiejętności techniczne.

Format odpowiedzi: lista umiejętności oddzielona przecinkami, bez dodatkowych wyjaśnień.

Przykład: Python, PyTorch, scikit-learn, time series forecasting, MLflow, Docker, AWS SageMaker

CV:
{cv_text}

Umiejętności:
"""

# Wywołanie OpenAI do ekstrakcji
from openai import OpenAI
response = openai_client.chat.completions.create(
    model="gpt-4o-mini",  # Tańszy model do ekstrakcji
    messages=[
        {"role": "user", "content": extraction_prompt}
    ],
    max_tokens=300,
    temperature=0  # Deterministyczna odpowiedź
)

# Wyciągnij wynik
extracted_skills = response.choices[0].message.content.strip()

print(f"✅ Wyekstraktowane umiejętności:")
print("-" * 60)
print(extracted_skills)
print("-" * 60)
print(f"\nDługość: {len(extracted_skills)} znaków")
print(f"(vs pełne CV: {len(cv_text)} znaków)")

🤖 EKSTRAKCJA SKILLS PRZEZ OpenAI
✅ Wyekstraktowane umiejętności:
------------------------------------------------------------
Python, PyTorch, scikit-learn, XGBoost, TensorFlow, SQL, Git, GitHub, GitHub Actions, regression, classification, clustering, time series forecasting, Prophet, ARIMA, hyperparameter tuning, GridSearchCV, RMSE, MAE, AUC, F1, ETL, ML pipelines, PySpark, AWS, S3, Athena, SageMaker, Airflow, MLflow, Docker, REST APIs, Jupyter, pandas, NumPy, Matplotlib, Seaborn, Plotly, feature engineering, EDA, anomaly detection, statistical analysis
------------------------------------------------------------

Długość: 434 znaków
(vs pełne CV: 9668 znaków)


In [12]:
# Generowanie embeddingu dla wyekstraktowanych skills
print("🔄 NOWE PORÓWNANIE - CV SKILLS vs OFERTY")
print("=" * 60)

# Wygeneruj embedding dla skills
cv_skills_embedding = calculate_embedding(extracted_skills)

if cv_skills_embedding is not None:
    print(f"✅ Embedding skills wygenerowany!\n")
    
    # Oblicz podobieństwo dla każdej oferty
    print("🎯 NOWE WYNIKI DOPASOWANIA:")
    print("-" * 60)
    
    for job in sample_jobs:
        # Oblicz podobieństwo
        similarity = cosine_similarity([cv_skills_embedding], [job['embedding']])[0][0]
        
        # Zapisz nowy wynik
        job['similarity_skills'] = similarity
        
        # Kolorowanie
        if similarity > 0.8:
            color = "🟢"
            rating = "Świetne dopasowanie"
        elif similarity > 0.6:
            color = "🟠"
            rating = "Średnie dopasowanie"
        else:
            color = "🔴"
            rating = "Słabe dopasowanie"
        
        # Porównaj ze starym wynikiem
        old_sim = job['similarity'] * 100
        new_sim = similarity * 100
        diff = new_sim - old_sim
        
        print(f"{color} [{job['id']}] {job['title'][:35]:<35} | {new_sim:>5.2f}% | {rating}")
        print(f"     (było: {old_sim:.2f}%, zmiana: {diff:+.2f}%)")
else:
    print("❌ Nie udało się wygenerować embeddingu")

🔄 NOWE PORÓWNANIE - CV SKILLS vs OFERTY
✅ Embedding skills wygenerowany!

🎯 NOWE WYNIKI DOPASOWANIA:
------------------------------------------------------------
🔴 [1] Senior Machine Learning Engineer    | 56.29% | Słabe dopasowanie
     (było: 43.87%, zmiana: +12.42%)
🔴 [2] Data Scientist                      | 44.91% | Słabe dopasowanie
     (było: 38.92%, zmiana: +5.99%)
🔴 [3] Python Backend Developer            | 45.01% | Słabe dopasowanie
     (było: 38.37%, zmiana: +6.64%)
🔴 [4] AI Research Engineer                | 45.65% | Słabe dopasowanie
     (było: 44.02%, zmiana: +1.63%)
🔴 [5] Junior Python Developer             | 32.67% | Słabe dopasowanie
     (było: 33.90%, zmiana: -1.23%)


In [13]:
# ROZBUDOWANE opisy ofert - bardziej szczegółowe
detailed_jobs = [
    {
        "id": 1,
        "title": "Senior Machine Learning Engineer",
        "company": "TechCorp",
        "description": """
        Senior ML Engineer position focused on predictive modeling and time series forecasting.
        Work with Python, PyTorch, scikit-learn on production ML systems.
        Build ETL pipelines, feature engineering, hyperparameter tuning (GridSearchCV).
        Deploy models using MLflow, Docker, AWS SageMaker.
        Time series: Prophet, ARIMA, anomaly detection.
        Metrics: RMSE, MAE, F1, AUC.
        Tools: Jupyter, pandas, NumPy, Git, SQL.
        Collaborate in Agile teams, 5+ years experience.
        """,
        "location": "Kraków, Poland",
        "link": "https://example.com/job1"
    },
    {
        "id": 2,
        "title": "Data Scientist",
        "company": "FinanceAI",
        "description": """
        Data Scientist for business intelligence and statistical analysis.
        Python, SQL, Tableau for reporting and dashboards.
        Statistical modeling, regression, clustering.
        ETL processes, data cleaning, EDA.
        No deep learning required - focus on business metrics.
        Tools: pandas, Matplotlib, Excel.
        """,
        "location": "Warsaw, Poland",
        "link": "https://example.com/job2"
    },
    {
        "id": 3,
        "title": "Python Backend Developer",
        "company": "WebDev Inc",
        "description": """
        Backend developer for web applications.
        Django, Flask, REST APIs, PostgreSQL.
        Docker, Git, CI/CD pipelines.
        No ML/AI - pure backend engineering.
        Microservices architecture.
        """,
        "location": "Remote",
        "link": "https://example.com/job3"
    },
    {
        "id": 4,
        "title": "AI Research Engineer",
        "company": "DeepMind Clone",
        "description": """
        Research ML Engineer position.
        PyTorch, TensorFlow, transformers, NLP.
        Deep learning research, publishing papers.
        PhD or MSc required.
        Cutting-edge AI, not production systems.
        Experimentation focus.
        """,
        "location": "London, UK",
        "link": "https://example.com/job4"
    },
    {
        "id": 5,
        "title": "Junior Python Developer",
        "company": "StartupXYZ",
        "description": """
        Entry-level Python position.
        Learn basics: Python syntax, Git, simple scripts.
        No ML, no advanced topics.
        Training provided.
        """,
        "location": "Gdańsk, Poland",
        "link": "https://example.com/job5"
    }
]

print("📋 ROZBUDOWANE OFERTY PRACY")
print("=" * 60)

# Wygeneruj embeddingi dla nowych opisów
for job in detailed_jobs:
    job_text = f"{job['title']}. {job['description']}"
    job['embedding'] = calculate_embedding(job_text)
    print(f"✅ [{job['id']}] {job['title'][:40]:<40} | {len(job_text):>4} znaków")

print("\n✅ Nowe embeddingi gotowe!")

📋 ROZBUDOWANE OFERTY PRACY
✅ [1] Senior Machine Learning Engineer         |  559 znaków
✅ [2] Data Scientist                           |  361 znaków
✅ [3] Python Backend Developer                 |  248 znaków
✅ [4] AI Research Engineer                     |  278 znaków
✅ [5] Junior Python Developer                  |  191 znaków

✅ Nowe embeddingi gotowe!


In [14]:
# FINALNE PORÓWNANIE - skills vs detailed jobs
print("🏆 FINALNE PORÓWNANIE - CV SKILLS vs ROZBUDOWANE OFERTY")
print("=" * 70)

results = []

for job in detailed_jobs:
    # Oblicz podobieństwo
    similarity = cosine_similarity([cv_skills_embedding], [job['embedding']])[0][0]
    
    # Zapisz wynik
    job['final_similarity'] = similarity
    results.append({
        'id': job['id'],
        'title': job['title'],
        'company': job['company'],
        'location': job['location'],
        'similarity': similarity,
        'link': job['link']
    })

# Sortuj od najlepszego dopasowania
results_sorted = sorted(results, key=lambda x: x['similarity'], reverse=True)

# Wyświetl wyniki
print("\n🎯 RANKING DOPASOWANIA:\n")

for rank, result in enumerate(results_sorted, 1):
    sim = result['similarity'] * 100
    
    # Kolorowanie
    if sim > 80:
        color = "🟢"
        rating = "Świetne dopasowanie"
    elif sim > 60:
        color = "🟠"
        rating = "Średnie dopasowanie"
    else:
        color = "🔴"
        rating = "Słabe dopasowanie"
    
    medal = "🥇" if rank == 1 else "🥈" if rank == 2 else "🥉" if rank == 3 else "  "
    
    print(f"{medal} {color} #{rank} | {sim:>5.2f}% | {result['title']}")
    print(f"       {result['company']} - {result['location']}")
    print(f"       {rating}")
    print()

print("=" * 70)
print(f"✅ TOP 3 rekomendacje dla Twojego profilu!")

🏆 FINALNE PORÓWNANIE - CV SKILLS vs ROZBUDOWANE OFERTY

🎯 RANKING DOPASOWANIA:

🥇 🟠 #1 | 61.37% | Senior Machine Learning Engineer
       TechCorp - Kraków, Poland
       Średnie dopasowanie

🥈 🔴 #2 | 49.76% | AI Research Engineer
       DeepMind Clone - London, UK
       Słabe dopasowanie

🥉 🔴 #3 | 49.53% | Data Scientist
       FinanceAI - Warsaw, Poland
       Słabe dopasowanie

   🔴 #4 | 49.10% | Python Backend Developer
       WebDev Inc - Remote
       Słabe dopasowanie

   🔴 #5 | 44.70% | Junior Python Developer
       StartupXYZ - Gdańsk, Poland
       Słabe dopasowanie

✅ TOP 3 rekomendacje dla Twojego profilu!


In [15]:
# PODSUMOWANIE EKSPERYMENTU
print("=" * 70)
print("📊 PODSUMOWANIE EKSPERYMENTU CV JOB MATCHER")
print("=" * 70)

print("\n✅ CO ZADZIAŁAŁO:")
print("  1. Ekstrakcja tekstu z PDF (pdfplumber)")
print("  2. Generowanie embeddingów OpenAI (text-embedding-3-small)")
print("  3. Ekstrakcja skills przez GPT-4o-mini")
print("  4. Cosine similarity dla matchingu")
print("  5. Ranking ofert od najlepszego dopasowania")

print("\n📈 WYNIKI:")
print(f"  🥇 Najlepsze dopasowanie: {results_sorted[0]['title']} ({results_sorted[0]['similarity']*100:.2f}%)")
print(f"  🥈 Drugie miejsce: {results_sorted[1]['title']} ({results_sorted[1]['similarity']*100:.2f}%)")
print(f"  🥉 Trzecie miejsce: {results_sorted[2]['title']} ({results_sorted[2]['similarity']*100:.2f}%)")

print("\n💡 WNIOSKI:")
print("  • Embeddingi działają najlepiej z podobnymi długościami tekstów")
print("  • Ekstrakcja skills (AI) > pełne CV dla matchingu")
print("  • Wyniki 60%+ = świetne dopasowanie (nie oczekuj 90%+)")
print("  • Im bardziej szczegółowe opisy ofert, tym lepsze wyniki")

print("\n🎯 NASTĘPNE KROKI DLA MVP:")
print("  1. Zapisać funkcje do utils/embeddings.py")
print("  2. Zapisać sample_jobs do data/sample_jobs.json")
print("  3. Zbudować aplikację Streamlit (app.py)")
print("  4. Dodać Azure Blob Storage dla uploadowanych CV")

print("\n" + "=" * 70)

📊 PODSUMOWANIE EKSPERYMENTU CV JOB MATCHER

✅ CO ZADZIAŁAŁO:
  1. Ekstrakcja tekstu z PDF (pdfplumber)
  2. Generowanie embeddingów OpenAI (text-embedding-3-small)
  3. Ekstrakcja skills przez GPT-4o-mini
  4. Cosine similarity dla matchingu
  5. Ranking ofert od najlepszego dopasowania

📈 WYNIKI:
  🥇 Najlepsze dopasowanie: Senior Machine Learning Engineer (61.37%)
  🥈 Drugie miejsce: AI Research Engineer (49.76%)
  🥉 Trzecie miejsce: Data Scientist (49.53%)

💡 WNIOSKI:
  • Embeddingi działają najlepiej z podobnymi długościami tekstów
  • Ekstrakcja skills (AI) > pełne CV dla matchingu
  • Wyniki 60%+ = świetne dopasowanie (nie oczekuj 90%+)
  • Im bardziej szczegółowe opisy ofert, tym lepsze wyniki

🎯 NASTĘPNE KROKI DLA MVP:
  1. Zapisać funkcje do utils/embeddings.py
  2. Zapisać sample_jobs do data/sample_jobs.json
  3. Zbudować aplikację Streamlit (app.py)
  4. Dodać Azure Blob Storage dla uploadowanych CV

